In [182]:
# import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd

In [183]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\c1264\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


### Visit the NASA Mars News Site

In [184]:
# visit the mars nasa news site
url = 'https://redplanetscience.com/'
browser.visit(url)

# optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [185]:
# set up Html parser
html = browser.html
news_soup = soup(html,'html.parser')
slide_elem = news_soup.select_one('div.list_text') # look for <div class=list_text>, select_one: return the first matching element

In [186]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find('div',class_='content_title').get_text()
news_title

"How NASA's Perseverance Mars Team Adjusted to Work in the Time of Coronavirus "

In [187]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div',class_='article_teaser_body').get_text()
news_p

'Like much of the rest of the world, the Mars rover team is pushing forward with its mission-critical work while putting the health and safety of their colleagues and community first.'

### JPL Space Images Featured Image

In [188]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [189]:
# find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [190]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [191]:
# find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars2.jpg'

In [192]:
# Use the base URL to create an absolute URL
img_url =f'https://spaceimages-mars.com/{img_url_rel}'
print(img_url)

https://spaceimages-mars.com/image/featured/mars2.jpg


### Mars Facts

In [193]:
# using pd.read_html to read HTML table as dataframe
df= pd.read_html('https://galaxyfacts-mars.com/')[0]
df

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [194]:
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace= True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [195]:
# add the DataFrame to a web application > convert to html
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

# D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

### Hemispheres

In [196]:
# 1. Use browser to visit the URL 
url = 'https://marshemispheres.com/'
browser.visit(url)

html = browser.html
hemi_soup = soup(html, 'html.parser')

In [199]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []

# 3. Write code to retrieve the image urls and titles for each hemisphere.
hemisphere = hemi_soup.find_all('div', class_='description')

for x in range(len(hemisphere)):
    hemispheres = {}
    browser.find_by_css('img.thumb')[x].click()
    
    # retrieve the data
    title = browser.find_by_css('h2.title').text
    url = browser.links.find_by_text("Sample")['href']
    
    # save data as dictionary
    hemispheres['img_url'] = url
    hemispheres['title'] = title
    
    # append the dictionary data in the hemispheres list
    hemisphere_image_urls.append(hemispheres)
    
    browser.back()

In [200]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls

[{'img_url': 'https://marshemispheres.com/images/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [201]:
# end the automated browsing session
browser.quit()